In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit


In [ ]:
def my_round(X , DX) :
    Ndecimal = np.int(np.ceil(np.abs(np.log10(DX))))
    DX = np.around( DX , Ndecimal )
    X = np.around( X , Ndecimal )
    return X , DX

def affine(x,a,b) :
    return a * x + b

def lineaire(x,a) :
    return a * x

##########################################################
##########################################################
##### 1ere partie Vitesse de corrosion a l abandon : 
##########################################################
##########################################################

In [ ]:
# Donnees exp : 

F = 96485
Mmolzn = 65.4

Minit = 34.8274 
Mfinal = 34.8218
t = 24 * 60 + 30 

Merr = 0.0001
terr = 1. 

# Calcul : 

DeltaM = Minit - Mfinal

Vcorr = DeltaM / t 

icorr = 2 * (F/Mmolzn) * (DeltaM / t )

# Propag incertitudes : 

DeltaMerr = np.sqrt(Merr**2 + Merr**2)

Vcorrerr = Vcorr * np.sqrt((DeltaMerr/DeltaM)**2 + (terr/t)**2 )
icorrerr = icorr * np.sqrt((DeltaMerr/DeltaM)**2 + (terr/t)**2 )

VRseul, VRseulerr = my_round(Vcorr , Vcorrerr)
iRseul, iRseulerr = my_round(icorr , icorrerr)



print ('Vitesse de corrosion : '+str(VRseul)+' +/- '+str(VRseulerr)+' g/s' )
print ('Courant de corrosion : '+str(iRseul)+' +/- '+str(iRseulerr)+' (A)' )



In [ ]:
##########################################################
##########################################################
##### 2eme partie Vitesse de corrosion Fe/Zn : 
##########################################################
##########################################################

In [ ]:
ls -lrt

In [ ]:
# Nom du catalogue
cat = 'znfe.csv'

In [ ]:
# Formatage du fichier csv le cas echeant : 
catinit= open(cat,'r').read()
print (catinit)



In [ ]:
catinit=catinit.replace(',',' ') #Delimiter 
#catinit=catinit.replace(',','.') #Virgule en point

cat = open('rdy2go.txt','w')
cat.write(catinit)
cat.close()

In [ ]:
# Lecture du fichier Formate 
Cat = np.genfromtxt('rdy2go.txt',dtype=[('i',np.float) , ('i_err',np.float) , ('Vzn',np.float) , ('Vzn_err',np.float) , ('Vfe',np.float) , ('Vfe_err',np.float) , ('R' , np.float) , ('R_err' , np.float)],skip_header=2)
Cat = Cat[np.invert(Cat['i']==0.)] # On retire les i=0 pour eviter les problemes avec ln(i)
Cat['i'] = Cat['i']/1000. # On passe de mA a A 
Cat['i_err'] = Cat['i_err']/1000. # On passe de mA a A 

In [ ]:
# Constante 
ECS = 0.2412

In [ ]:
# Correction des potentiel : 
Cat['Vzn'] = Cat['Vzn'] - ECS
Cat['Vfe'] = Cat['Vfe'] - ECS

In [ ]:
# Preparation des array pour les fits : 
Vzn = Cat['Vzn']
Vznerr = Cat['Vzn_err']
lnizn = np.log( np.abs(Cat['i']) )
lniznerr = Cat['i_err']/np.abs(Cat['i'])

Vfe = Cat['Vfe']
Vfeerr = Cat['Vfe_err']
lnife = np.log( np.abs(Cat['i']) )
lnifeerr = Cat['i_err']/np.abs(Cat['i'])


In [ ]:
# Selection des domaines : 
%matplotlib notebook
plt.figure()

plt.xlabel('ln(|i|)')
plt.ylabel('E (V)')

plt.errorbar( lnizn , Vzn, xerr=lniznerr , yerr=Vznerr , color='tomato', fmt = 'o',markersize=4, label='Zn')
plt.errorbar( lnife , Vfe, xerr=lnifeerr , yerr=Vfeerr , color='royalblue', fmt ='s',markersize=4, label='Fe')
plt.legend()
plt.show()


In [ ]:
print('Pour le Zn : ')

xminZn = np.float(input('Xmin '))
xmaxZn = np.float(input('Xmax '))

booli = lnizn>xminZn
lnizn = lnizn[booli]
lniznerr = lniznerr[booli]
Vzn = Vzn[booli]
Vznerr = Vznerr[booli]


booli = lnizn<xmaxZn
lnizn = lnizn[booli]
lniznerr = lniznerr[booli]
Vzn = Vzn[booli]
Vznerr = Vznerr[booli]


print('Pour le Fe : ')

xminFe = np.float(input('Xmin '))
xmaxFe = np.float(input('Xmax '))

booli = lnife>xminFe
lnife = lnife[booli]
lnifeerr = lnifeerr[booli]
Vfe = Vfe[booli]
Vfeerr = Vfeerr[booli]

booli = lnife<xmaxFe
lnife = lnife[booli]
lnifeerr = lnifeerr[booli]
Vfe = Vfe[booli]
Vfeerr = Vfeerr[booli]


In [ ]:
# Regression lineaire : 
poptzn, pcovzn = curve_fit(affine , lnizn , Vzn, sigma = Vznerr) 
znerr = np.sqrt(np.diag(pcovzn))

poptfe, pcovfe = curve_fit(affine , lnife , Vfe , sigma = Vznerr)
ferr = np.sqrt(np.diag(pcovfe))


In [ ]:
# Calcul Courant de corrosion et Potentiel Mixte (lnicorr ; icorr ; Em)

lnicorr = (poptzn[1] - poptfe[1]) / (poptfe[0] - poptzn[0]) # Croisement des deux regressions
Em = poptfe[0] * lnicorr  + poptfe[1]
icorr = np.exp(lnicorr)
vcorr = (Mmolzn/(2*F)) * icorr


In [ ]:
# Incertitude et propagation : 

# En posant lni = A/B : 
A = (poptzn[1] - poptfe[1])
DA = np.sqrt(ferr[1]**2 + znerr[1]**2)

B = (poptfe[0] - poptzn[0])
DB = np.sqrt(ferr[0]**2 + znerr[0]**2)

Dlnicorr = lnicorr * np.sqrt( (DA/A)**2 + (DB/B)**2) ### Incertitude sur ln(icorr)

Dicorr = np.abs(icorr) * np.abs(Dlnicorr) ### Incertitude de icorr

Dvcorr = (Mmolzn/(2*F)) * Dicorr

C = poptfe[0] * lnicorr
DC = C * np.sqrt( (ferr[0]/poptfe[0])**2 + (Dlnicorr/lnicorr)**2  )

DEm =  np.sqrt ( DC**2 + ferr[1]**2 ) ### Incertitude sur Em




In [ ]:
####### Plot resultats : 

xmin = np.min([np.min(lnife),np.min(lnizn)])

DeltaX = np.abs(lnicorr - xmin)

X = np.arange(xmin-0.1*DeltaX , lnicorr + 0.3*DeltaX,0.1)

plt.figure(2) 

plt.errorbar( lnife , Vfe, xerr=lnifeerr , yerr=Vfeerr , color='royalblue', fmt ='s',markersize=4, label='Fe')
plt.plot( X , poptfe[0]*X + poptfe[1], 'royalblue')

plt.errorbar( lnizn , Vzn, xerr=lniznerr , yerr=Vznerr , color='tomato', fmt = 'o',markersize=4, label='Zn')
plt.plot( X , poptzn[0]*X + poptzn[1], 'tomato')


ymin = np.min([np.min(poptfe[0]*X + poptfe[1]),np.min(poptzn[0]*X + poptzn[1])])
ymax = np.max([np.max(poptfe[0]*X + poptfe[1]),np.max(poptzn[0]*X + poptzn[1])])

plt.xlim(xmin-0.2*DeltaX , lnicorr + 0.4*DeltaX) 

DeltaY = ymax-ymin

plt.ylim(ymin-0.1*DeltaY , ymax+0.1*DeltaY)

print ('###############' )
print (r'Em = '+str(Em)+' +/- '+str(DEm)+' (V)')
print (r'ic = '+str(icorr)+' +/- '+str(Dicorr)+' (A)')

EmR , DEmR = my_round(Em , DEm)
icorrR , DicorrR = my_round(icorr , Dicorr)
vcorrR , DvcorrR = my_round(vcorr , Dvcorr)

ratio = np.int(np.ceil(vcorrR / VRseul))

print (r'vc (lié) = '+str(vcorrR)+' +/- '+str(DvcorrR)+' (g/s)')
print ('______________________________________________')
print (r'vc (seul) = '+str(VRseul)+' +/- '+str(VRseulerr)+' (g/s)')
print (r'ratio (vlie/vseul) ~ '+str(ratio))




#if np.abs(ymax)>np.abs(ymin) : plt.ylim(0.9*ymin , 1.1*ymax)
#else : plt.ylim(1.1*ymin , 0.9*ymax)

plt.plot ([ xmin-0.2*DeltaX , lnicorr ] , [ Em , Em ], 'black', linestyle='dashed')
plt.plot ([ lnicorr , lnicorr ] , [ ymin-0.1*DeltaY , Em ], 'black', linestyle='dashed')

plt.text ( xmin ,Em  , r'$E_m = $'+str(EmR)+' +/- '+str(DEmR)+' V' , fontsize =16 )
plt.text ( lnicorr ,ymin  , r'$|i_{corr}| = $'+str(icorrR)+' +/- '+str(DicorrR)+' A' , fontsize =16 )


plt.xlabel(r'$ln\, |i| \ (A)$')
plt.ylabel(r'$E\ (V)$')
plt.show()




##########################################################
##########################################################
######    3 eme partie : Remonter a la resistance ########
######          interne du circuit                ########
##########################################################
##########################################################


In [ ]:
U = Cat['Vfe'] - Cat['Vzn'] - Cat['R']*Cat['i']
# En posant C = R I 

Cat['R'][Cat['R']==0.] = 10**-6 # On enleve les R=0 pour eviter les / par 0
DC = Cat['R']*Cat['i'] * np.sqrt( (Cat['R_err']/Cat['R'])**2 + (Cat['i_err']/Cat['i'])**2 )
Uerr = np.sqrt(Cat['Vfe_err']**2 + Cat['Vzn_err']**2 + DC**2 )

plt.figure()


plt.errorbar( Cat['i'] , U, xerr=Cat['i_err'] , yerr=Uerr , color='royalblue', fmt ='o',markersize=4)

poptR, pcovR = curve_fit(lineaire , Cat['i'] , U , sigma=Uerr )

R = poptR[0]
DR = np.sqrt(np.diag(pcovR))[0]

Rr , DRr = my_round(R , DR)

plt.plot(Cat['i'] , poptR[0] * Cat['i'], color = 'darkred') 

plt.xlabel(r'$i\ (A)$')
plt.ylabel(r'$U = V_{Fe} - V_{Zn} - R_{var} \, i \ = \ R_{int} \, i \ (V)$')
print ('###############')
print (r'Rint = '+str(Rr)+' +/- '+str(DRr)+' (Ohm)')

plt.show()